# Кибериммунная автономность$\\$Создание конструктивно защищённого автономного наземного транспортного средства$\\$Модуль 4

## О документе

Версия 1.03

Модуль 4 для регионального этапа соревнований по кибериммунной автономности

### Модуль 4. Следование по трассе с киберпрепятствиями

Для успешного выполнения этого задания необходимое активировать специальный режим работы виртуальной машинки - в рамках этого задания можно менять только блоки, отвечающие за безопасность (ограничитель, монитор безопасности), **другие блоки менять запрещено**.

При прохождении маршрута будут имитироваться атаки со стороны злоумышленников, которые будут пытаться нарушить цели безопасности. Важно, чтобы им это не удалось.

1. Запустите свою машинку и убедитесь, что она проходит всю трассу без нарушений ограничений скорости. При необходимости измените логику работы блока безопасности
2. Добавьте контроль доставки груза в модуле SafetyBlock - убедитесь, что груз доставляется до конечной точки маршрута.
3. Если в модуле 3 вы реализовали монитор безопасности - не забудьте его перенести в этот модуль, это принесёт дополнительные баллы!

Активация киберпрепятствий в системе управления:

после инициализации системы управления добавьте следующую строку
```python
control_system.enable_surprises()
```

В этом блоке добавьте все ваши реализации изменённых бортовых систем

In [1]:
# ваш код

Если у вас настроена и работает СУПА, установите в True значение переменной afcs_present

In [2]:
afcs_present = True

Поменяем идентификатор машинки для этого модуля

In [3]:
car_id = "m4"

В следующем блоке измените маршрут на ваш, его можно скопировать из модуля 2.
 
Для проверки работы систем безопасности будут активированы киберпрепятствия

```python
control_system.enable_surprises()
```

In [ ]:
# используем то же маршрутное задание, которое было в модуле 2
from time import sleep

from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.mission_planner import Mission
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer
from src.wpl_parser import WPLParser
from src.mission_type import GeoSpecificSpeedLimit


# для примера возьмём маршрут из модуля 2
# следует заменить на маршрут, требуемый по регламенту!
wpl_file = "module2.wpl"

parser = WPLParser(wpl_file)    
points = parser.parse()

# обновите скоростные ограничения для вашего маршрута!
speed_limits = [
    GeoSpecificSpeedLimit(0, 30),
    GeoSpecificSpeedLimit(1, 60),
    GeoSpecificSpeedLimit(5, 20),
]


home = points[0]
mission = Mission(home=home, waypoints=points,speed_limits=speed_limits, armed=True)

# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 


# создание блоков передачи данных в СУПА
if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

# создание основных функциональных блоков
mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)

communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)

navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

safety_block = SafetyBlock(queues_dir=queues_dir, log_level=LOG_INFO)


# сборка всех запускаемых блоков в одном "кузове"
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block
    ] if afcs_present else [
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block
    ])

#################################
# АКТИВАЦИЯ КИБЕРПРЕПЯТСТВИЙ
control_system.enable_surprises()
#################################

# запуск всех блоков
system_components.start()

# ограничение поездки по времени
# в случае превышения времени выполнения ячейки на более чем 10 секунд от заданного, 
# допустимо перезапустить вычислительное ядро и повторно выполнить весь блокнот, штрафные очки за это не начисляются
# при условии, что повторный запуск закончился успешно
sleep(100)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][QUEUES] регистрируем очередь safety
[ИНФО][SAFETY] создан ограничитель
[ИНФО][MISSION PLANNER] старт системы планирования заданий
[ИНФО][CARGO] старт блока грузового отсека
[ИНФО][CONTROL] старт системы управления
[ИНФО][SAFETY] старт ограничителя
[ИНФО][MISSION PLANNER] запрошена новая задача, отправляем получателям
[ИНФО][MISSION PLANNER] новая задача отправлена в коммуни

[ИНФО][CONTROL] установлена новая задача, начинаем следовать по маршруту, текущее время 10:30:34.262205
[ИНФО][CARGO] заблокировать грузовой отсек
[ИНФО][CARGO] грузовой отсек заблокирован
[ИНФО][CONTROL] новая скорость 30 (была 0)
[ИНФО][CONTROL] новое направление 251 (было 0)
[ИНФО][SAFETY] сегмент пройден
[ИНФО][CONTROL] сегмент пройден
[ИНФО][CONTROL] новая скорость 60 (была 30)
[ИНФО][CONTROL] новое направление 142 (было 251)
[ОШИБКА][SAFETY] принудительная установка направления! Запрошенное направление 142.15592535488207, расчётное 322.1559253548821
[ОШИБКА][SAFETY] принудительная установка направления! Запрошенное направление 142.5982455662425, расчётное 322.5982455662425
[ИНФО][CONTROL] новое направление 143 (было 142)
[ОШИБКА][SAFETY] принудительная установка направления! Запрошенное направление 143.04282516926833, расчётное 323.0428251692683
[ОШИБКА][SAFETY] принудительная установка направления! Запрошенное направление 143.48955414368152, расчётное 323.4895541436815
[ОШИБКА][

Убедитесь, что 
1. ваша машинка успешно прошла весь заданный маршрут
2. не превысила ограничения скорости
3. успешно доставила груз

Если всё так - поздравляем, вы справились с заданием! Обязательно зафиксируйте все изменения в репозитории!



Мы будем признательны за обратную связь - любые комментарии, которые вы можете дать по итогам выполнения этого задания. 

Например, 

- было ли задание понятным по шкале 1..10 (1 - ничего не понятно, 10 - вопросов вообще не было, всё понятно)?
- было ли задание интересным по шкале 1..10 (1 - скука смертная, 10 - лучшее, что вам пока встречалось на олимпиадах)? 
- что бы вы предложили изменить, чтобы сделает его более интересным?
- по шкале 1..10 насколько сложным оно было для вас?
- что было самым трудным в задании? 

Авторы наиболее развёрнутых и интересных комментариев получат особенный приз от Лаборатории Касперского!

Дополнительная информация о кибериммунной разработке
- https://os.kaspersky.ru/cyber-immune-development/ 
- https://github.com/sergey-sobolev/cyberimmune-systems/wiki/%D0%9A%D0%B8%D0%B1%D0%B5%D1%80%D0%B8%D0%BC%D0%BC%D1%83%D0%BD%D0%B8%D1%82%D0%B5%D1%82
- канал в телеграм: https://t.me/learning_cyberimmunity
